# 📊 Multi-Dataset Merger (v2) - Student Dropout Prediction
## Combining 5 Datasets for Maximum Accuracy

**Updates in v2:**
- **Smart Feature Mapping:** Maps different column names (e.g., 'Medu' -> 'Mother_Qualification') to a standard set.
- **Deduplication:** Removes duplicate records if datasets are identical.
- **Robust Loading:** Handles different CSV separators.
- **Target Standardization:** Unifies target values to 'Dropout', 'Graduate', 'Enrolled'.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Add parent directory to path
sys.path.append('..')
import config

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2. Load All Datasets

In [2]:
# Load each dataset with smart separator detection
datasets = {}

dataset_info = [
    ('dataset1', config.DATASET_1_PATH),
    ('dataset2', config.DATASET_2_PATH),
    ('dataset3', config.DATASET_3_PATH),
    ('dataset4', config.DATASET_4_PATH),
    ('dataset5', config.DATASET_5_PATH)
]

for name, path in dataset_info:
    try:
        # Try loading with default comma separator
        df = pd.read_csv(path)
        
        # If only 1 column found, try semicolon separator
        if df.shape[1] == 1:
            df = pd.read_csv(path, sep=';')
            
        datasets[name] = df
        print(f"✓ {config.DATASET_NAMES[name]:40} - {df.shape[0]:5,} rows × {df.shape[1]:2} columns")
    except FileNotFoundError:
        print(f"✗ {config.DATASET_NAMES[name]:40} - FILE NOT FOUND")
    except Exception as e:
        print(f"✗ {config.DATASET_NAMES[name]:40} - ERROR: {str(e)}")

print(f"\n📊 Total datasets loaded: {len(datasets)}")

✓ Higher Education Predictors              - 4,424 rows × 35 columns
✓ Student Performance                      -   395 rows × 33 columns
✓ Academic Success                         - 4,424 rows × 37 columns
✓ Mental Health                            -   101 rows × 11 columns
✓ Predict Dropout & Success                - 4,424 rows × 37 columns

📊 Total datasets loaded: 5


## 3. Standardize Target Variables

In [3]:
def standardize_target(df, dataset_name=""):
    """
    Standardize target variable to: Dropout, Graduate, Enrolled
    """
    df_copy = df.copy()
    target_col = None
    
    # --- SPECIAL CASES ---
    
    # Case 1: Student Performance Dataset (Uses G3 grades)
    if 'G3' in df_copy.columns:
        print(f"  ℹ️ Detected 'G3' (Grade) column. Creating Target from grades.")
        # Rule: G3 < 10 is Fail (Dropout), G3 >= 10 is Pass (Graduate)
        df_copy['Target'] = df_copy['G3'].apply(lambda x: 'Dropout' if x < 10 else 'Graduate')
        return df_copy

    # Case 2: Mental Health Dataset (No direct target)
    if 'Do you have Depression?' in df_copy.columns and 'Target' not in df_copy.columns:
        print(f"  ⚠️ Dataset '{dataset_name}' appears to be the Mental Health dataset with no Dropout target.")
        print(f"  ⚠️ It will be excluded from training but used for analysis.")
        return df_copy

    # --- GENERAL DETECTION ---
    if target_col is None:
        if 'Target' in df_copy.columns:
            target_col = 'Target'
        elif 'target' in df_copy.columns:
            target_col = 'target'
        else:
            target_candidates = ['Dropout', 'Status', 'dropout', 'status']
            for candidate in target_candidates:
                matches = [col for col in df_copy.columns if candidate in col]
                matches = [m for m in matches if 'marital' not in m.lower()]
                matches = [m for m in matches if 'Pstatus' not in m]
                if matches:
                    target_col = matches[0]
                    break
    
    if target_col is None:
        print(f"⚠️ No target column found. Skipping...")
        return df_copy
    
    print(f"  Identified target column: '{target_col}'")
    
    if target_col != 'Target':
        if 'Target' in df_copy.columns:
            df_copy = df_copy.drop(columns=['Target'])
        df_copy = df_copy.rename(columns={target_col: 'Target'})
    
    # Standardize values
    df_copy['Target'] = df_copy['Target'].astype(str).str.strip()
    return df_copy

standardized_datasets = {}
for name, df in datasets.items():
    print(f"\nProcessing {config.DATASET_NAMES[name]}...")
    standardized_datasets[name] = standardize_target(df, config.DATASET_NAMES[name])
    if 'Target' in standardized_datasets[name].columns:
        print(f"✓ Target standardized. Values: {standardized_datasets[name]['Target'].unique()[:5]}")
    else:
        print(f"✗ No target found")


Processing Higher Education Predictors...
  Identified target column: 'Target'
✓ Target standardized. Values: ['Dropout' 'Graduate' 'Enrolled']

Processing Student Performance...
  ℹ️ Detected 'G3' (Grade) column. Creating Target from grades.
✓ Target standardized. Values: ['Dropout' 'Graduate']

Processing Academic Success...
  Identified target column: 'Target'
✓ Target standardized. Values: ['Dropout' 'Graduate' 'Enrolled']

Processing Mental Health...
  ⚠️ Dataset 'Mental Health' appears to be the Mental Health dataset with no Dropout target.
  ⚠️ It will be excluded from training but used for analysis.
✗ No target found

Processing Predict Dropout & Success...
  Identified target column: 'Target'
✓ Target standardized. Values: ['Dropout' 'Graduate' 'Enrolled']


## 4. Advanced Feature Harmonization (Mapping)

In [4]:
# Define Standard Feature Names
STANDARD_FEATURES = [
    'Age',
    'Gender',
    'Marital_Status',
    'Course',
    'Mother_Qualification',
    'Father_Qualification',
    'Previous_Qualification',
    'Admission_Grade',
    'Displaced',
    'Debtor',
    'Tuition_Fees_Up_To_Date',
    'Scholarship_Holder',
    'Unemployment_Rate',
    'Inflation_Rate',
    'GDP',
    'Target'
]

# Define Mappings for specific dataset types
def get_mapping(df_columns):
    mapping = {}
    cols = [c.lower() for c in df_columns]
    
    # 1. Academic Dataset (Datasets 1, 3, 5)
    if 'mother\'s qualification' in cols or 'mother qualification' in cols:
        mapping = {
            'Age at enrollment': 'Age',
            'Gender': 'Gender',
            'Marital status': 'Marital_Status',
            'Course': 'Course',
            'Mother\'s qualification': 'Mother_Qualification',
            'Father\'s qualification': 'Father_Qualification',
            'Previous qualification': 'Previous_Qualification',
            'Admission grade': 'Admission_Grade',
            'Displaced': 'Displaced',
            'Debtor': 'Debtor',
            'Tuition fees up to date': 'Tuition_Fees_Up_To_Date',
            'Scholarship holder': 'Scholarship_Holder',
            'Unemployment rate': 'Unemployment_Rate',
            'Inflation rate': 'Inflation_Rate',
            'GDP': 'GDP',
            'Target': 'Target'
        }
    
    # 2. Student Performance Dataset (Dataset 2)
    elif 'medu' in cols and 'fedu' in cols:
        mapping = {
            'age': 'Age',
            'sex': 'Gender',
            'Medu': 'Mother_Qualification',
            'Fedu': 'Father_Qualification',
            'address': 'Displaced', # Proxy: U/R might correlate
            'paid': 'Tuition_Fees_Up_To_Date', # Proxy
            'Target': 'Target'
        }
        
    # 3. Mental Health Dataset (Dataset 4)
    elif 'choose your gender' in cols:
        mapping = {
            'Age': 'Age',
            'Choose your gender': 'Gender',
            'What is your course?': 'Course',
            'Marital status': 'Marital_Status'
        }
        
    return mapping

def harmonize_dataset(df, dataset_name):
    # Get appropriate mapping
    mapping = get_mapping(df.columns)
    
    # Create new dataframe with standard columns
    new_df = pd.DataFrame()
    new_df['Dataset_Source'] = [dataset_name] * len(df)
    
    # Apply mapping
    for original_col, standard_col in mapping.items():
        if original_col in df.columns:
            new_df[standard_col] = df[original_col]
    
    # Handle Gender Normalization (Male/Female -> 1/0)
    if 'Gender' in new_df.columns:
        # If string 'M'/'F' or 'Male'/'Female'
        if new_df['Gender'].dtype == 'O':
            new_df['Gender'] = new_df['Gender'].apply(lambda x: 1 if str(x).lower().startswith('m') else 0)

    # Handle Displaced Normalization (U/R -> 1/0 or Yes/No -> 1/0)
    if 'Displaced' in new_df.columns:
        if new_df['Displaced'].dtype == 'O':
            # Map 'U' (Urban) to 1, 'R' (Rural) to 0? Or 'Yes'/'No'?
            # Assuming Displaced means "Living away from home".
            # If 'address' (U/R) was mapped here, U=Urban, R=Rural.
            # Let's assume U=1, R=0 for now, or check values.
            # Safest is to map 'yes'/'no' if present.
            new_df['Displaced'] = new_df['Displaced'].apply(lambda x: 1 if str(x).lower() in ['yes', 'u', 'urban', '1'] else 0)

    # Handle Tuition Fees Normalization (Yes/No -> 1/0)
    if 'Tuition_Fees_Up_To_Date' in new_df.columns:
        if new_df['Tuition_Fees_Up_To_Date'].dtype == 'O':
            new_df['Tuition_Fees_Up_To_Date'] = new_df['Tuition_Fees_Up_To_Date'].apply(lambda x: 1 if str(x).lower() in ['yes', 'paid', '1'] else 0)
            
    # Handle Scholarship Holder Normalization (Yes/No -> 1/0)
    if 'Scholarship_Holder' in new_df.columns:
        if new_df['Scholarship_Holder'].dtype == 'O':
            new_df['Scholarship_Holder'] = new_df['Scholarship_Holder'].apply(lambda x: 1 if str(x).lower() in ['yes', '1'] else 0)
            
    # Handle Debtor Normalization (Yes/No -> 1/0)
    if 'Debtor' in new_df.columns:
        if new_df['Debtor'].dtype == 'O':
            new_df['Debtor'] = new_df['Debtor'].apply(lambda x: 1 if str(x).lower() in ['yes', '1'] else 0)
            
    # Fill missing standard columns with NaN
    for col in STANDARD_FEATURES:
        if col not in new_df.columns:
            new_df[col] = np.nan
            
    return new_df

print("Harmonizing and Mapping Features...\n")
harmonized_list = []
for name, df in standardized_datasets.items():
    if 'Target' in df.columns:  # Only include datasets with targets
        harmonized_df = harmonize_dataset(df, config.DATASET_NAMES[name])
        harmonized_list.append(harmonized_df)
        print(f"✓ {config.DATASET_NAMES[name]:40} - Mapped {harmonized_df.shape[1]} features")
    else:
        print(f"- {config.DATASET_NAMES[name]:40} - Skipped (No Target)")

Harmonizing and Mapping Features...

✓ Higher Education Predictors              - Mapped 17 features
✓ Student Performance                      - Mapped 17 features
✓ Academic Success                         - Mapped 17 features
- Mental Health                            - Skipped (No Target)
✓ Predict Dropout & Success                - Mapped 17 features


## 5. Merge and Deduplicate

In [5]:
# Concatenate
merged_df = pd.concat(harmonized_list, ignore_index=True)
initial_count = len(merged_df)

# Drop Duplicates (Ignoring Source column)
# This handles the case where Dataset 1, 3, and 5 are identical
cols_to_check = [c for c in merged_df.columns if c != 'Dataset_Source']
merged_df = merged_df.drop_duplicates(subset=cols_to_check)
final_count = len(merged_df)

print(f"\nInitial Record Count: {initial_count:,}")
print(f"Duplicates Removed:   {initial_count - final_count:,}")
print(f"Final Record Count:   {final_count:,}")

# Save
output_path = config.MERGED_DATASET_PATH
merged_df.to_csv(output_path, index=False)
print(f"\n✓ Saved to {output_path}")


Initial Record Count: 13,667
Duplicates Removed:   4,654
Final Record Count:   9,013



✓ Saved to D:\Santosh_minor\notebooks\..\data\processed\merged_datasets.csv


In [6]:
# Validation
print("\nMissing Values (%):")
print((merged_df[STANDARD_FEATURES].isnull().sum() / len(merged_df) * 100).sort_values(ascending=False))

if 'Target' in merged_df.columns:
    print("\nTarget Distribution:")
    print(merged_df['Target'].value_counts())


Missing Values (%):
Admission_Grade            50.926440
Course                      3.139909
Previous_Qualification      3.139909
Marital_Status              3.139909
Unemployment_Rate           3.139909
Inflation_Rate              3.139909
Scholarship_Holder          3.139909
Debtor                      3.139909
GDP                         3.139909
Father_Qualification        0.000000
Gender                      0.000000
Age                         0.000000
Mother_Qualification        0.000000
Displaced                   0.000000
Tuition_Fees_Up_To_Date     0.000000
Target                      0.000000
dtype: float64

Target Distribution:
Target
Graduate    4493
Dropout     2939
Enrolled    1581
Name: count, dtype: int64
